# Interactive Analysis Tool

## User Guide

This notebook provides an easy-to-use interface for running calculations based on temperature and flow inputs.

### Instructions:
1. Run the setup cell below by clicking the ▶️ button or pressing Shift+Enter
2. Enter your values for temperatures (T1-T4) and flows (F1-F2) in the input fields
3. Click the "Run Calculation" button to process the data
4. View the results and visualizations below the button

**Note:** If you encounter any errors, please ensure all input values are numbers.

In [8]:
# Import required libraries
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import io
import base64
import math
from datetime import datetime
from pathlib import Path
import os

# Set up plotting configuration
%matplotlib inline
plt.style.use('ggplot')



In [2]:
# Configuration constants (normally these might come from a CSV or configuration file)
# These are hidden from the end user but accessible to the calculations
REFERENCE_CONSTANTS = {
    'alpha': 0.0023,
    'beta': 1.456,
    'gamma': 0.876,
    'delta': 2.345,
    'min_temp': 20.0,
    'max_temp': 50.0,
    'min_flow': 0.1,
    'max_flow': 100.0,
    'min_flow_F': 1000,
    'max_flow_F': 10000,
    'reference_materials': [
        {'id': 1, 'name': 'Material A', 'coefficient': 0.45},
        {'id': 2, 'name': 'Material B', 'coefficient': 0.72},
        {'id': 3, 'name': 'Material C', 'coefficient': 0.91}
    ]
}

# Create reference dataframe (simulating data that might come from a CSV)
reference_data = pd.DataFrame({
    'temp_range': ['10-30', '31-60', '61-90', '91-120'],
    'efficiency_factor': [0.82, 0.91, 0.86, 0.75],
    'correction_factor': [1.05, 1.00, 1.02, 1.15]
})


In [3]:
# Dictionary to store all dataframes
csv_data = {}

# Function to get a specific dataframe
def get_dataframe(csv_name):
    """
    Get a specific dataframe by name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    
    Returns:
    pandas.DataFrame: The dataframe, or None if not found
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    return csv_data[csv_name]

def load_csv_files(data_dir="Data"):
    """
    Load all CSV files from the specified directory.
    
    Parameters:
    data_dir (str): Path to the directory containing CSV files
    
    Returns:
    dict: Dictionary of dataframes with normalized names as keys
    """
    global csv_data
    
    try:
        # Get all CSV files in the directory
        csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        
        # Load each CSV file into a dataframe
        for file in csv_files:
            # Create a normalized name for the dataframe (without .csv extension, uppercase)
            df_name = os.path.splitext(file)[0].upper()
            file_path = os.path.join(data_dir, file)
            
            try:
                # Try to read the CSV file
                csv_data[df_name] = pd.read_csv(file_path)
                print(f"Loaded: {file} as {df_name}")
            except Exception as e:
                # Try with different separators if automatic detection fails
                try:
                    csv_data[df_name] = pd.read_csv(file_path, sep=';')
                    print(f"Loaded: {file} as {df_name} (using semicolon separator)")
                except:
                    try:
                        csv_data[df_name] = pd.read_csv(file_path, sep='\t')
                        print(f"Loaded: {file} as {df_name} (using tab separator)")
                    except Exception as e2:
                        print(f"Failed to load {file}: {e2}")
        
        return csv_data
    
    except Exception as e:
        print(f"Error loading CSV files: {e}")
        return {}

def get_lookup_value(csv_name, lookup_value, col_index_lookup=0, col_index_return=1):
    """
    Look up a value in a CSV file based on finding the first value 
    in col_index_lookup that is >= lookup_value, then return the 
    corresponding value from col_index_return.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against col_index_lookup)
    col_index_lookup (int): Index of column to search in (default: 0)
    col_index_return (int/str/list): Index of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from col_index_return corresponding to the first row where
    col_index_lookup >= lookup_value, or None if not found.
    If col_index_return is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[col_index_lookup] if isinstance(col_index_lookup, int) else col_index_lookup
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(col_index_return, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in col_index_return:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[col_index_return] if isinstance(col_index_return, int) else col_index_return
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def get_lookup_value_by_name(csv_name, lookup_value, lookup_col_name, return_col_name):
    """
    Look up a value in a CSV file based on finding the first value 
    in lookup_col_name that is >= lookup_value, then return the 
    corresponding value from return_col_name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against lookup_col_name)
    lookup_col_name (str): Name of column to search in
    return_col_name (str/list): Name of column(s) to return value from.
                               Can be a single column name or list of names.
    
    Returns:
    The value from return_col_name corresponding to the first row where
    lookup_col_name >= lookup_value, or None if not found.
    If return_col_name is a list, returns a dictionary with column names as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Check if lookup column exists
    if lookup_col_name not in df.columns:
        print(f"Column '{lookup_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
        return None
    
    # If return_col_name is a list, check all columns exist
    if isinstance(return_col_name, (list, tuple)):
        for col in return_col_name:
            if col not in df.columns:
                print(f"Column '{col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
                # Continue anyway, will just skip this column
    else:
        # Single column - check it exists
        if return_col_name not in df.columns:
            print(f"Column '{return_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col_name)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col_name] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_name, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_name:
            if col in df.columns:
                result[col] = matched_row[col]
        return result
    else:
        # Return a single column
        return matched_row[return_col_name]

def get_exact_match(csv_name, lookup_value, lookup_col_index=0, return_col_index=1):
    """
    Look up a value in a CSV file based on finding an exact match
    in lookup_col_index, then return the corresponding value from return_col_index.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (must match exactly)
    lookup_col_index (int/str): Index or name of column to search in (default: 0)
    return_col_index (int/str/list): Index or name of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from return_col_index corresponding to the row where
    lookup_col_index == lookup_value, or None if not found.
    If return_col_index is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[lookup_col_index] if isinstance(lookup_col_index, int) else lookup_col_index
    
    # Find rows where lookup column == lookup_value
    matching_rows = df[df[lookup_col] == lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_index, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_index:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[return_col_index] if isinstance(return_col_index, int) else return_col_index
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def show_csv_info(csv_name=None):
    """
    Display information about loaded CSV files.
    
    Parameters:
    csv_name (str, optional): Name of specific CSV to display info for.
                             If None, displays info for all CSVs.
    """
    global csv_data
    
    if csv_name:
        # Normalize the CSV name
        csv_name = csv_name.upper()
        
        # Check if the CSV has been loaded
        if csv_name not in csv_data:
            print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
            return
        
        # Display info for the specified CSV
        df = csv_data[csv_name]
        print(f"\n=== CSV: {csv_name} ===")
        print(f"Shape: {df.shape}")
        print("\nColumns:")
        for i, col in enumerate(df.columns):
            print(f"  {i}: {col}")
        print("\nFirst 5 rows:")
        print(df.head())
    else:
        # Display info for all CSVs
        for name, df in csv_data.items():
            print(f"\n=== CSV: {name} ===")
            print(f"Shape: {df.shape}")
            print("\nColumns:")
            for i, col in enumerate(df.columns):
                print(f"  {i}: {col}")
            print("\nFirst 5 rows:")
            print(df.head())

# Example usage
# First load all CSVs
# data_dir = "Data"  # Update this to your data directory
# load_csv_files(data_dir)

# Example of how to use the lookup functions
# pipe_size = 2.0
# pipe_cost = get_lookup_value("PIPCOST", pipe_size)
# print(f"Cost for pipe size {pipe_size}: {pipe_cost}")

# Example of how to use exact match lookup
# valve_type = "GATE"
# valve_cost = get_exact_match("CVALV", valve_type)
# print(f"Cost for valve type {valve_type}: {valve_cost}")

# Example of calculations using values from CSVs
# def calculate_system_cost(pipe_size, pipe_length, valve_type=None):
#     # Get pipe cost per unit length
#     pipe_cost_per_unit = get_lookup_value("PIPCOST", pipe_size)
#     total_pipe_cost = pipe_cost_per_unit * pipe_length
#     
#     # Add valve cost if specified
#     total_valve_cost = 0
#     if valve_type:
#         valve_cost = get_exact_match("CVALV", valve_type)
#         if valve_cost:
#             total_valve_cost = valve_cost
#     
#     # Calculate total cost
#     total_cost = total_pipe_cost + total_valve_cost
#     return total_cost
# 
# # Calculate system cost
# system_cost = calculate_system_cost(2.0, 100, "GATE")
# print(f"Total system cost: ${system_cost:.2f}")

load_csv_files()
# show_csv_info()

Loaded: ALLHX.csv as ALLHX
Loaded: CVALV.csv as CVALV
Loaded: HX.csv as HX
Loaded: IVALV.csv as IVALV
Loaded: JOINTS.csv as JOINTS
Loaded: MW Price Data.csv as MW PRICE DATA
Loaded: PIPCOST.csv as PIPCOST
Loaded: PIPSZ.csv as PIPSZ
Loaded: ROOM.csv as ROOM


{'ALLHX':      wha  T1 itdt  T2 TCSapp     F1  T4  T3     F2 FWSapp  ... Unnamed: 15  \
 0    1.0  20   10  30      2  1,493  18  28  1,440      2  ...         NaN   
 1    1.0  20   10  30      3  1,493  17  27  1,440      3  ...         NaN   
 2    1.0  20   10  30      5  1,493  15  25  1,440      5  ...         NaN   
 3    1.0  20   12  32      2  1,244  18  30  1,200      2  ...         NaN   
 4    1.0  20   12  32      3  1,244  17  29  1,200      3  ...         NaN   
 ..   ...  ..  ...  ..    ...    ...  ..  ..    ...    ...  ...         ...   
 131  5.0  45   12  57      5  6,209  40  52  6,061      5  ...         NaN   
 132  5.0  45   14  59      2  5,322  43  57  5,195      2  ...         NaN   
 133  5.0  45   14  59      3  5,322  42  56  5,195      3  ...         NaN   
 134  5.0  45   14  59      5  5,322  40  54  5,195      5  ...         NaN   
 135    A   B   DT   C     D       Z   Z   Z      Z      Z  ...         NaN   
 
     Unnamed: 16 Unnamed: 17 Unnamed: 18 

In [4]:
# Formulas
def mw_divd(F1, T1, T2):
    mw_value = get_MW(F1, T1, T2)
    result = mw_value / 1000000
    rounded = round(result, 2)
    return rounded


def get_itdt(T1,T2):
    # Temperature Difference Across IT side of Heat Exchanger
    x=T2-T1
    # print(x)
    return T2-T1

def get_PipeSize_Suggested(F1):
    x=get_lookup_value("PIPSZ",F1)
    return x

def get_MW(F1, T1,T2):
    # F1 = l/m
    # l/m / 60 seconds * 4186 J/kg * itdt
    itdt = get_itdt(T1,T2)
    formula = F1 / 60 * 4186 * itdt
    return formula

def get_PipeCost_perMeter(F1):
    pss = get_PipeSize_Suggested(F1)
    pcpm = get_lookup_value("PIPCOST",pss,1)
    return pcpm
    
def get_PipeLength(F1,T1,T2):
    mw = mw_divd(F1, T1, T2)
    pipelength = get_lookup_value("ROOM",mw)    
    return pipelength

def get_PipeCost_Total(F1,T1,T2):
    ttl=get_PipeCost_perMeter(F1)*get_PipeLength(F1,T1,T2)
    return ttl
    

In [5]:
# Helper functions for calculations

def get_efficiency_factor(temp):
    """Get efficiency factor based on temperature range"""
    if temp >= 30:
        return reference_data.iloc[0]['efficiency_factor']
    elif temp >= 60:
        return reference_data.iloc[1]['efficiency_factor']
    elif temp >= 90:
        return reference_data.iloc[2]['efficiency_factor']
    else:
        return reference_data.iloc[3]['efficiency_factor']

def get_correction_factor(temp):
    """Get correction factor based on temperature range"""
    if temp >= 30:
        return reference_data.iloc[0]['correction_factor']
    elif temp >= 60:
        return reference_data.iloc[1]['correction_factor']
    elif temp >= 90:
        return reference_data.iloc[2]['correction_factor']
    else:
        return reference_data.iloc[3]['correction_factor']

def calculate_heat_transfer(t1, t2, flow):
    """Calculate heat transfer based on temperature difference and flow"""
    alpha = REFERENCE_CONSTANTS['alpha']
    beta = REFERENCE_CONSTANTS['beta']
    return alpha * flow * beta * abs(t2 - t1)

def calculate_efficiency(t1, t2, t3, t4, f1, f2):
    """Calculate system efficiency based on all parameters"""
    gamma = REFERENCE_CONSTANTS['gamma']
    delta = REFERENCE_CONSTANTS['delta']
    
    avg_temp = (t1 + t2 + t3 + t4) / 4
    eff_factor = get_efficiency_factor(avg_temp)
    corr_factor = get_correction_factor(avg_temp)
    
    input_energy = calculate_heat_transfer(t1, t2, f1)
    output_energy = calculate_heat_transfer(t3, t4, f2)
    
    if input_energy == 0:
        return 0
    
    base_efficiency = (output_energy / input_energy) * eff_factor * corr_factor
    adjusted_efficiency = base_efficiency * gamma * (1 + delta / (avg_temp + 10))
    
    return min(adjusted_efficiency, 1.0)  # Efficiency cannot be > 100%

def generate_performance_rating(efficiency):
    """Generate a performance rating based on calculated efficiency"""
    if efficiency >= 0.90:
        return "Excellent"
    elif efficiency >= 0.80:
        return "Very Good"
    elif efficiency >= 0.70:
        return "Good"
    elif efficiency >= 0.60:
        return "Fair"
    else:
        return "Poor"




In [ ]:
# Validation Functions
def validate_inputs(t1, t2, t3, t4, f1, f2):
    """Validate that all inputs are within acceptable ranges"""
    errors = []
    min_temp = REFERENCE_CONSTANTS['min_temp']
    max_temp = REFERENCE_CONSTANTS['max_temp']
    min_flow = REFERENCE_CONSTANTS['min_flow']
    max_flow = REFERENCE_CONSTANTS['max_flow']
    min_flow_F = REFERENCE_CONSTANTS['min_flow_F']
    max_flow_F = REFERENCE_CONSTANTS['max_flow_F']
    
    for name, value, min_val, max_val in [
        ('T1', t1, min_temp, max_temp),
        ('T2', t2, min_temp, max_temp),
        ('T3', t3, min_temp, max_temp),
        ('T4', t4, min_temp, max_temp),
        ('F1', f1, min_flow_F, max_flow_F),
        ('F2', f2, min_flow_F, max_flow_F)
    ]:
        if value < min_val:
            errors.append(f"{name} is too low (minimum: {min_val})")
        elif value > max_val:
            errors.append(f"{name} is too high (maximum: {max_val})")
    
    return errors

In [6]:
# Functions for graphical elements
def fig_to_base64(fig):
    """Convert a matplotlib figure to a base64 encoded image"""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

def create_charts(t1, t2, t3, t4, f1, f2, efficiency):
    """Create visualization charts for the results"""
    # Create a figure with multiple subplots
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Temperature comparison chart
    temps = ['T1', 'T2', 'T3', 'T4']
    temp_values = [t1, t2, t3, t4]
    axs[0, 0].bar(temps, temp_values, color=['#ff9999', '#ff9999', '#66b3ff', '#66b3ff'])
    axs[0, 0].set_title('Temperature Readings')
    axs[0, 0].set_ylabel('Temperature (°C)')
    for i, v in enumerate(temp_values):
        axs[0, 0].text(i, v + 1, f"{v:.1f}", ha='center')
    
    # 2. Flow comparison chart
    flows = ['F1', 'F2']
    flow_values = [f1, f2]
    axs[0, 1].bar(flows, flow_values, color=['#99ff99', '#ffcc99'])
    axs[0, 1].set_title('Flow Measurements')
    axs[0, 1].set_ylabel('Flow Rate (units)')
    for i, v in enumerate(flow_values):
        axs[0, 1].text(i, v + 0.5, f"{v:.2f}", ha='center')
    
    # 3. Efficiency gauge chart (simplified)
    theta = np.linspace(0, np.pi, 100)
    radius = 1
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    
    axs[1, 0].plot(x, y, 'k-', linewidth=2)
    axs[1, 0].fill_between(x, 0, y, color='#eeeeee')
    
    # Add efficiency needle
    needle_angle = np.pi * efficiency
    axs[1, 0].plot([0, radius * np.cos(needle_angle)], [0, radius * np.sin(needle_angle)], 'r-', linewidth=3)
    
    # Add efficiency percentage text
    axs[1, 0].text(0, 0, f"{efficiency*100:.1f}%", ha='center', va='center', fontsize=20, fontweight='bold')
    
    # Format the gauge chart
    axs[1, 0].set_xlim(-1.2, 1.2)
    axs[1, 0].set_ylim(-0.2, 1.2)
    axs[1, 0].axis('equal')
    axs[1, 0].axis('off')
    axs[1, 0].set_title('System Efficiency')
    
    # 4. Heat transfer comparison
    input_heat = calculate_heat_transfer(t1, t2, f1)
    output_heat = calculate_heat_transfer(t3, t4, f2)
    
    heat_labels = ['Input Heat', 'Output Heat']
    heat_values = [input_heat, output_heat]
    
    axs[1, 1].bar(heat_labels, heat_values, color=['#ffcc99', '#99ccff'])
    axs[1, 1].set_title('Heat Transfer Comparison')
    axs[1, 1].set_ylabel('Heat Transfer (units)')
    for i, v in enumerate(heat_values):
        axs[1, 1].text(i, v + max(heat_values)*0.05, f"{v:.1f}", ha='center')
    
    plt.tight_layout()
    
    return fig

def format_results_html(t1, t2, t3, t4, f1, f2):
    """Format calculation results as an HTML report"""
    # Perform calculations
    itdt = get_itdt(t1,t2)
    f=get_PipeSize_Suggested(f1)
    mw = get_MW(f1, t1, t2)
    pcpm = get_PipeCost_perMeter(f1)
    pl = get_PipeLength(f1,t1,t2)
    pss = get_PipeSize_Suggested(f1)
    ttl = int(get_PipeCost_Total(f1,t1,t2))
    print("itdt: ", itdt)
    print("f: ", f)
    print("mw: ", mw)
    print("pss: ", pss)
    print ("pcpm: ", pcpm)
    print ("pl: ", pl)
    print("")
    print("Total: €", ttl)
    
    efficiency = calculate_efficiency(t1, t2, t3, t4, f1, f2)
    performance_rating = generate_performance_rating(efficiency)
    input_heat = calculate_heat_transfer(t1, t2, f1)
    output_heat = calculate_heat_transfer(t3, t4, f2)
    avg_temp = (t1 + t2 + t3 + t4) / 4
    eff_factor = get_efficiency_factor(avg_temp)
    corr_factor = get_correction_factor(avg_temp)
    
    # Create charts
    fig = create_charts(t1, t2, t3, t4, f1, f2, efficiency)
    chart_img = fig_to_base64(fig)
    plt.close(fig)  # Close the figure to free memory
    
    # Generate timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create HTML report
    html = f"""
    <div style="font-family: Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; background-color: #f9f9f9; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1);">
        <h2 style="color: #333; border-bottom: 2px solid #ddd; padding-bottom: 10px;">Calculation Results</h2>
        <p style="color: #777; font-style: italic;">Generated: {timestamp}</p>
        
        <div style="display: flex; justify-content: space-between; margin: 20px 0;">
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; margin-right: 10px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Input Parameters</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">T1:</td><td style="padding: 5px;">{t1} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T2:</td><td style="padding: 5px;">{t2} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T3:</td><td style="padding: 5px;">{t3} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T4:</td><td style="padding: 5px;">{t4} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">F1:</td><td style="padding: 5px;">{f1} units</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">F2:</td><td style="padding: 5px;">{f2} units</td></tr>
                </table>
            </div>
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Key Results</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">System Efficiency:</td>
                        <td style="padding: 5px;"><span style="font-size: 18px; font-weight: bold; color: #4CAF50;">{efficiency*100:.1f}%</span></td>
                    </tr>
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">Performance Rating:</td>
                        <td style="padding: 5px;"><span style="font-weight: bold;">{performance_rating}</span></td>
                    </tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Input Heat Transfer:</td><td style="padding: 5px;">{input_heat:.2f} units</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Output Heat Transfer:</td><td style="padding: 5px;">{output_heat:.2f} units</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Average Temperature:</td><td style="padding: 5px;">{avg_temp:.1f} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Efficiency Factor:</td><td style="padding: 5px;">{eff_factor:.2f}</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Correction Factor:</td><td style="padding: 5px;">{corr_factor:.2f}</td></tr>
                </table>
            </div>
        </div>
        
        <div style="background-color: white; padding: 15px; border-radius: 5px; margin-top: 20px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
            <h3 style="color: #444; margin-top: 0;">Visualizations</h3>
            <img src="data:image/png;base64,{chart_img}" style="width: 100%; max-width: 800px; display: block; margin: 0 auto;">
        </div>
        
        <div style="margin-top: 20px; font-size: 12px; color: #777; text-align: center;">
            Analysis performed using the Interactive Analysis Tool
        </div>
    </div>
    """
    
    return html

In [7]:
# Create the user interface

# Title and style for the input form
display(HTML("""
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
    <h3 style="margin-top: 0;">Parameter Input Form</h3>
    <p style="margin-bottom: 0;">Enter values for temperatures and flows, then click "Run Calculation"</p>
</div>
"""))

# Create input widgets with helpful descriptions
style = {'description_width': '120px'}
layout = widgets.Layout(width='400px')

t1_widget = widgets.FloatText(
    value=31.0,
    description='T1 (Outlet to TCS Temperature):',
    tooltip='Temperature at Outlet to TCS (°C)',
    style=style, layout=layout
)

t2_widget = widgets.FloatText(
    value=41.9,
    description='T2 (Inlet from TCS Temperature):',
    tooltip='Temperature at Inlet from TCS (°C)',
    style=style, layout=layout
)

t3_widget = widgets.FloatText(
    value=35,
    description='T3 (Outlet to Consumer Temperature):',
    tooltip='Temperature at Outlet to Consumer (°C)',
    style=style, layout=layout
)

t4_widget = widgets.FloatText(
    value=25,
    description='T4 (Inlet from Consumer Temperature):',
    tooltip='Temperature at Inlet from Consumer (°C)',
    style=style, layout=layout
)

f1_widget = widgets.FloatText(
    value=2100,
    description='F1 (TCS Flow Rate):',
    tooltip='Flow rate to Data Hall (l/m)',
    style=style, layout=layout
)

f2_widget = widgets.FloatText(
    value=2000,
    description='F2 (FWS Flow Rate):',
    tooltip='Flow rate to Consumer (l/m)',
    style=style, layout=layout
)

# Create output area for results and error messages
output_area = widgets.Output()

# Run calculation button with styling
button = widgets.Button(
    description='Run Calculation',
    button_style='success',
    tooltip='Click to process the inputs and generate results',
    icon='calculator',
    layout=widgets.Layout(width='200px', height='40px')
)

# Function to handle button click
def on_button_click(b):
    with output_area:
        clear_output()
        
        # Get values from widgets
        try:
            t1 = float(t1_widget.value)
            t2 = float(t2_widget.value)
            t3 = float(t3_widget.value)
            t4 = float(t4_widget.value)
            f1 = float(f1_widget.value)
            f2 = float(f2_widget.value)
        except ValueError:
            display(HTML("""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Error:</strong> All values must be numbers. Please check your inputs.
            </div>
            """))
            return
        
        # Validate inputs
        errors = validate_inputs(t1, t2, t3, t4, f1, f2)
        if errors:
            error_list = "<br>".join([f"• {error}" for error in errors])
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Input Validation Errors:</strong><br>
                {error_list}
            </div>
            """))
            return
        
        # Generate and display results directly, without the intermediate display
        try:
            html_results = format_results_html(t1, t2, t3, t4, f1, f2)
            display(HTML(html_results))
            # output_area.append_display_data(HTML(html_results))
        except Exception as e:
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Calculation Error:</strong><br>
                An error occurred during calculation: {str(e)}
            </div>
            """))
            
# Connect the button click handler
button.on_click(on_button_click)

# Display the widgets in a organized layout
input_box = widgets.VBox([
    widgets.HBox([widgets.VBox([t1_widget, t2_widget, t3_widget]), widgets.VBox([t4_widget, f1_widget, f2_widget])]),
    widgets.HBox([button]),
    output_area
], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px 0'))

display(input_box)

# THIS IS NEW GENERATED CODE NOT YET IMPLEMENTED, FROM CURSOR, FOR CHANGING THE INTERFACE

# Create the user interface

# Title and style for the input form
display(HTML("""
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
    <h3 style="margin-top: 0;">Parameter Input Form</h3>
    <p style="margin-bottom: 0;">Enter values for the required parameters, then click "Run Calculation"</p>
</div>
"""))

# Create input widgets with helpful descriptions
style = {'description_width': '120px'}
layout = widgets.Layout(width='400px')

t1_widget = widgets.FloatText(
    value=31.0,
    description='T1 (Outlet to TCS Temperature):',
    tooltip='Temperature at Outlet to TCS (°C)',
    style=style, layout=layout
)

tcs_wha_widget = widgets.FloatText(
    value=41.9,
    description='TCS Wha:',
    tooltip='TCS Wha value',
    style=style, layout=layout
)

delta_t_widget = widgets.FloatText(
    value=35,
    description='Delta T:',
    tooltip='Temperature difference',
    style=style, layout=layout
)

tcs_approach_widget = widgets.FloatText(
    value=25,
    description='TCS Approach:',
    tooltip='TCS Approach value',
    style=style, layout=layout
)

# Create output area for results and error messages
output_area = widgets.Output()

# Run calculation button with styling
button = widgets.Button(
    description='Run Calculation',
    button_style='success',
    tooltip='Click to process the inputs and generate results',
    icon='calculator',
    layout=widgets.Layout(width='200px', height='40px')
)

# Updated validation function for new parameters
def validate_inputs(t1, tcs_wha, delta_t, tcs_approach):
    """Validate that all inputs are within acceptable ranges"""
    errors = []
    min_temp = REFERENCE_CONSTANTS['min_temp']
    max_temp = REFERENCE_CONSTANTS['max_temp']
    
    for name, value, min_val, max_val in [
        ('T1', t1, min_temp, max_temp),
        ('TCS Wha', tcs_wha, min_temp, max_temp),
        ('Delta T', delta_t, 0, max_temp),
        ('TCS Approach', tcs_approach, 0, max_temp)
    ]:
        if value < min_val:
            errors.append(f"{name} is too low (minimum: {min_val})")
        elif value > max_val:
            errors.append(f"{name} is too high (maximum: {max_val})")
    
    return errors

# Updated results formatting function
def format_results_html(t1, tcs_wha, delta_t, tcs_approach):
    """Format calculation results as an HTML report"""
    # Perform calculations
    itdt = get_itdt(t1, tcs_wha)
    mw = get_MW(tcs_wha, t1, tcs_wha)  # Using tcs_wha as flow rate
    pcpm = get_PipeCost_perMeter(tcs_wha)
    pl = get_PipeLength(tcs_wha, t1, tcs_wha)
    pss = get_PipeSize_Suggested(tcs_wha)
    ttl = int(get_PipeCost_Total(tcs_wha, t1, tcs_wha))
    
    # Calculate efficiency based on new parameters
    efficiency = calculate_efficiency(t1, tcs_wha, delta_t, tcs_approach, tcs_wha, tcs_wha)
    performance_rating = generate_performance_rating(efficiency)
    input_heat = calculate_heat_transfer(t1, tcs_wha, tcs_wha)
    output_heat = calculate_heat_transfer(delta_t, tcs_approach, tcs_wha)
    avg_temp = (t1 + tcs_wha + delta_t + tcs_approach) / 4
    eff_factor = get_efficiency_factor(avg_temp)
    corr_factor = get_correction_factor(avg_temp)
    
    # Create charts
    fig = create_charts(t1, tcs_wha, delta_t, tcs_approach, tcs_wha, tcs_wha, efficiency)
    chart_img = fig_to_base64(fig)
    plt.close(fig)  # Close the figure to free memory
    
    # Generate timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create HTML report
    html = f"""
    <div style="font-family: Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; background-color: #f9f9f9; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1);">
        <h2 style="color: #333; border-bottom: 2px solid #ddd; padding-bottom: 10px;">Calculation Results</h2>
        <p style="color: #777; font-style: italic;">Generated: {timestamp}</p>
        
        <div style="display: flex; justify-content: space-between; margin: 20px 0;">
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; margin-right: 10px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Input Parameters</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">T1:</td><td style="padding: 5px;">{t1} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">TCS Wha:</td><td style="padding: 5px;">{tcs_wha} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Delta T:</td><td style="padding: 5px;">{delta_t} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">TCS Approach:</td><td style="padding: 5px;">{tcs_approach} °C</td></tr>
                </table>
            </div>
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Key Results</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">System Efficiency:</td>
                        <td style="padding: 5px;"><span style="font-size: 18px; font-weight: bold; color: #4CAF50;">{efficiency*100:.1f}%</span></td>
                    </tr>
                    <tr>
                        <td style="padding: 5px; font-weight: bold;">Performance Rating:</td>
                        <td style="padding: 5px;"><span style="font-weight: bold;">{performance_rating}</span></td>
                    </tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Input Heat Transfer:</td><td style="padding: 5px;">{input_heat:.2f} units</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Output Heat Transfer:</td><td style="padding: 5px;">{output_heat:.2f} units</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Average Temperature:</td><td style="padding: 5px;">{avg_temp:.1f} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Efficiency Factor:</td><td style="padding: 5px;">{eff_factor:.2f}</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Correction Factor:</td><td style="padding: 5px;">{corr_factor:.2f}</td></tr>
                </table>
            </div>
        </div>
        
        <div style="background-color: white; padding: 15px; border-radius: 5px; margin-top: 20px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
            <h3 style="color: #444; margin-top: 0;">Visualizations</h3>
            <img src="data:image/png;base64,{chart_img}" style="width: 100%; max-width: 800px; display: block; margin: 0 auto;">
        </div>
        
        <div style="margin-top: 20px; font-size: 12px; color: #777; text-align: center;">
            Analysis performed using the Interactive Analysis Tool
        </div>
    </div>
    """
    
    return html

# Function to handle button click
def on_button_click(b):
    with output_area:
        clear_output()
        
        # Get values from widgets
        try:
            t1 = float(t1_widget.value)
            tcs_wha = float(tcs_wha_widget.value)
            delta_t = float(delta_t_widget.value)
            tcs_approach = float(tcs_approach_widget.value)
        except ValueError:
            display(HTML("""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Error:</strong> All values must be numbers. Please check your inputs.
            </div>
            """))
            return
        
        # Validate inputs
        errors = validate_inputs(t1, tcs_wha, delta_t, tcs_approach)
        if errors:
            error_list = "<br>".join([f"• {error}" for error in errors])
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Input Validation Errors:</strong><br>
                {error_list}
            </div>
            """))
            return
        
        # Generate and display results
        try:
            html_results = format_results_html(t1, tcs_wha, delta_t, tcs_approach)
            display(HTML(html_results))
        except Exception as e:
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Calculation Error:</strong><br>
                An error occurred during calculation: {str(e)}
            </div>
            """))
            
# Connect the button click handler
button.on_click(on_button_click)

# Display the widgets in a organized layout
input_box = widgets.VBox([
    widgets.HBox([widgets.VBox([t1_widget, tcs_wha_widget]), widgets.VBox([delta_t_widget, tcs_approach_widget])]),
    widgets.HBox([button]),
    output_area
], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px 0'))

display(input_box)


## Technical Documentation

### System Architecture
This notebook implements a simple calculation engine with the following components:

1. **Input Interface** - Interactive widgets for parameter entry
2. **Calculation Engine** - Functions for processing the input data
3. **Visualization Module** - Generation of charts and graphs
4. **Output Formatter** - HTML report generation

### Reference Data
The notebook uses embedded reference data that would normally come from external CSV files. This includes:
- Temperature-based efficiency factors
- Correction factors for different operational ranges
- System constants for calculations

### Calculation Logic
The primary calculations estimate:
1. Heat transfer between temperature points
2. System efficiency based on all parameters
3. Performance rating classification

### Maintenance Notes
To modify the reference data or calculation logic:
1. Edit the `REFERENCE_CONSTANTS` dictionary to change calculation parameters
2. Modify the `reference_data` DataFrame to update efficiency or correction factors
3. Update the calculation functions if the underlying formulas need to change